# 프로젝트 6 멋진 작사가 만들기

1. 라이브러리 로드

In [32]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import numpy as np
import tensorflow as tf
import os, re
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.8.2


2. 학습할 데이터 준비

In [33]:
txt_file_path = "/content/drive/MyDrive/data/lyrics/lyrics/*" #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])




데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


In [34]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [35]:
corpus = []

for sentence in raw_corpus:
    
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessd_sentence = preprocess_sentence(sentence)
    if len(preprocessd_sentence.split(' ')) > 15: continue
    corpus.append(preprocessd_sentence)
    
corpus[ : 10]


['<start> hook <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face verse <end>',
 '<start> somethin ain t right when we talkin <end>']

3. 모델 준비

In [36]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    

In [55]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,
        filters=' ',
        oov_token='<unk>'
    )

tokenizer.fit_on_texts(corpus)
tensor = tokenizer.texts_to_sequences(corpus)
tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

VOCAB_SIZE = tokenizer.num_words + 1   

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=12)

BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

embedding_size = 512
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

- num_words를 15000으로 늘림.  
- embedding_size와 hidden_size를 조정함.  

In [50]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [56]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[-1.37694122e-04,  3.64059597e-05,  6.41370134e-04, ...,
         -1.33131063e-04, -2.37313448e-04, -1.96396446e-04],
        [-1.61555756e-04, -5.66811214e-05,  6.31674193e-04, ...,
         -1.35644019e-04, -5.57060179e-04, -2.05239703e-04],
        [ 9.96904928e-05, -3.64086591e-05,  1.54908776e-04, ...,
         -8.93613178e-05, -4.07557003e-04, -2.93054909e-04],
        ...,
        [-8.98867322e-04, -6.79226476e-04,  3.90163448e-04, ...,
         -9.41253733e-04,  6.05904963e-04, -8.18234752e-04],
        [-9.90417087e-04, -3.84746876e-04,  2.12086918e-04, ...,
         -7.70659593e-04,  9.06850619e-04, -1.00526831e-03],
        [-1.05542596e-03, -4.25217440e-05, -2.08703568e-06, ...,
         -5.79724845e-04,  1.18510798e-03, -1.16411573e-03]],

       [[-1.37694122e-04,  3.64059597e-05,  6.41370134e-04, ...,
         -1.33131063e-04, -2.37313448e-04, -1.96396446e-04],
        [-2.23241019e-04, -1.09976863e-05,  7

4. 학습

In [57]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none') 
# 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.

lyricist.compile(loss=loss, optimizer=optimizer)

lyricist.fit(dataset, epochs=10)


Epoch 1/10
487/487 [==============================] - 6762s 14s/step - loss: 3.5623
Epoch 2/10
487/487 [==============================] - 6749s 14s/step - loss: 3.0409
Epoch 3/10
487/487 [==============================] - 6778s 14s/step - loss: 2.8648
Epoch 4/10
487/487 [==============================] - 6754s 14s/step - loss: 2.7362
Epoch 5/10
487/487 [==============================] - 6790s 14s/step - loss: 2.6271
Epoch 6/10
487/487 [==============================] - 6906s 14s/step - loss: 2.5269
Epoch 7/10
487/487 [==============================] - 6798s 14s/step - loss: 2.4334
Epoch 8/10
487/487 [==============================] - 6798s 14s/step - loss: 2.3436
Epoch 9/10
487/487 [==============================] - 6782s 14s/step - loss: 2.2575
Epoch 10/10
487/487 [==============================] - 6767s 14s/step - loss: 2.1748


In [60]:
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

5. 검증

In [61]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

6. 회고

- RNN의 구성을 확인하는 시간이었다. 시간순서에 따른 학습인데, 각 글자별 학습이 진행되어 학습시간이 엄청 길었다.  
- 학습 중간 중간에 모델을 저장하여 불러오고 싶었는데 keras에서 제공하는 기본 모델이 아니다보니 저장이 되지 않았다.  
- 분명히 논리적이고 통계적이겠으나, 아직은 테스트에 대한 결과를 가지고 검토를 할 수 없어서 이게 맞는가 싶다.  
- loss는 2.2이하로 맞췄으나 밤새 돌린 결과가 "<start> i love you , i love you <end>" 이렇게 심플할수가...